In [3]:
# from google.colab import drive
# import zipfile
# import os


# drive.mount('/content/drive')

# zip_path = '/content/drive/MyDrive/apredizajeautomatico/QuickDraw-Animals.zip'  # Cambia esta ruta
# zip_path_2 = '/content/drive/MyDrive/apredizajeautomatico/QuickDraw-10-Tarea2.zip'

# extract_path = '/content/imagenes_descomprimidas'
# os.makedirs(extract_path, exist_ok=True)

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# with zipfile.ZipFile(zip_path_2, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print(f"Archivos extraídos en: {extract_path}")

# print("Archivos extraídos:")
# print(os.listdir(extract_path))

In [4]:
# importacion de librerias
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


# Parte 1 - Lectura de Imágenes

In [5]:
def load_image_paths_and_labels(file_path):
    """Carga las rutas de las imágenes y las etiquetas desde un archivo de texto."""
    image_paths = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file.readlines():
            path, label = line.strip().split('\t')
            image_paths.append(path)
            labels.append(int(label))
    return image_paths, np.array(labels)

def load_images(image_paths, folder_route):
    """Carga las imágenes y las aplana a vectores."""
    images = []
    for path in image_paths:
        with Image.open(folder_route + path) as img:
            img_array = np.array(img).reshape(-1)
            images.append(img_array)
    return np.array(images)

QuickDraw-10

In [6]:
# # Cargar las rutas de entrenamiento y prueba junto con las etiquetas
# train_image_paths_10, train_labels_10 = load_image_paths_and_labels("/content/imagenes_descomprimidas/QuickDraw-10/train.txt")
# test_image_paths_10, test_labels_10 = load_image_paths_and_labels("/content/imagenes_descomprimidas/QuickDraw-10/test.txt")

# # Cargar y procesar las imágenes
# train_images_10 = load_images(train_image_paths_10, "/content/imagenes_descomprimidas/QuickDraw-10/")
# test_images_10 = load_images(test_image_paths_10, "/content/imagenes_descomprimidas/QuickDraw-10/")


# Cargar las rutas de entrenamiento y prueba junto con las etiquetas
train_image_paths_10, train_labels_10 = load_image_paths_and_labels("./QuickDraw-10/train.txt")
test_image_paths_10, test_labels_10 = load_image_paths_and_labels("./QuickDraw-10/test.txt")

# Cargar y procesar las imágenes
train_images_10 = load_images(train_image_paths_10, "./QuickDraw-10/")
test_images_10 = load_images(test_image_paths_10, "./QuickDraw-10/")

In [7]:
# Preparacion datos de validacion
train_images_10, val_image_10, train_labels_10, val_labels_10 = train_test_split(
    train_images_10, train_labels_10, test_size=0.15, random_state=42, stratify=train_labels_10
)

In [8]:
# normalizacion
train_images_10 = train_images_10.astype('float32') / 255.0
test_images_10 = test_images_10.astype('float32') / 255.0
val_image_10 = val_image_10.astype('float32') / 255.0

In [9]:
# one hot encoding
n_classes=10
train_labels_10 = to_categorical(train_labels_10, num_classes=n_classes)
test_labels_10 = to_categorical(test_labels_10, num_classes=n_classes)
val_labels_10 = to_categorical(val_labels_10, num_classes=n_classes)

QuickDraw-Animals

In [10]:
def load_images_and_labels(dataset):
    """
    Carga imágenes y etiquetas para train y test del dataset QuickDraw-Animals.

    Retorna:
      X_test, y_test, X_train, y_train (numpy arrays)
    """
    if dataset != "Animals":
        raise ValueError("Sólo soporta dataset 'Animals'.")

    # base_path = '/content/imagenes_descomprimidas/QuickDraw-Animals/'
    base_path = './QuickDraw-Animals'
    mapping_file = os.path.join(base_path, 'mapping.txt')

    # Leer mapping.txt y crear diccionario etiqueta->número
    label_map = {}
    with open(mapping_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                label = parts[0]
                idx = int(parts[1])
                label_map[label] = idx

    def load_images_from_folder(folder_path):
        images = []
        labels = []
        # Las subcarpetas son las clases
        for label_name in sorted(os.listdir(folder_path)):
            label_folder = os.path.join(folder_path, label_name)
            if os.path.isdir(label_folder) and label_name in label_map:
                for img_file in sorted(os.listdir(label_folder)):
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(label_folder, img_file)
                        with Image.open(img_path) as img:
                            img_array = np.array(img).reshape(-1)
                            images.append(img_array)
                            labels.append(label_map[label_name])
        return np.array(images), np.array(labels)

    # Cargar test
    # X_test, y_test = load_images_from_folder('/content/imagenes_descomprimidas/QuickDraw-Animals/test_images/test_images/')
    X_test, y_test = load_images_from_folder('./QuickDraw-Animals/test_images/test_images')

    # Cargar train
    # X_train, y_train = load_images_from_folder('/content/imagenes_descomprimidas/QuickDraw-Animals/train_images/train_images/')
    X_train, y_train = load_images_from_folder('./QuickDraw-Animals/train_images/train_images')


    return X_test, y_test, X_train, y_train


In [11]:
test_images_animals, test_labels_animals, train_images_animals, train_labels_animals = load_images_and_labels("Animals")

In [12]:
# Preparacion datos de validacion
train_images_animals, val_images_animals, train_labels_animals, val_labels_animals = train_test_split(
    train_images_animals, train_labels_animals, test_size=0.15, random_state=42, stratify=train_labels_animals
)

In [13]:
# normalizacion
train_images_animals = train_images_animals.astype('float32') / 255.0
test_images_animals = test_images_animals.astype('float32') / 255.0
val_images_animals = val_images_animals.astype('float32') / 255.0

In [14]:
# one hot encoding
n_classes=12
train_labels_animals = to_categorical(train_labels_animals, num_classes=n_classes)
test_labels_animals = to_categorical(test_labels_animals, num_classes=n_classes)
val_labels_animals = to_categorical(val_labels_animals, num_classes=n_classes)

## Parte 2 - Construcción de modelos

Clase MLP

In [15]:
class MLP(tf.keras.Model):
    # defining components
    def __init__(self, layers_size, n_classes, activation='sigmoid'):
        super(MLP, self).__init__()
        self.layer_list = []
        for lsize in layers_size:
            self.layer_list.append(tf.keras.layers.Dense(lsize))
        self.classifier = tf.keras.layers.Dense(n_classes)
        self.activation = activation


    # defining architecture
    def call(self, inputs):
        x = inputs
        for mlp_layer in self.layer_list:
            x = mlp_layer(x)
            if self.activation == 'sigmoid':
                x = tf.keras.activations.sigmoid(x)
            elif self.activation == 'relu':
                x = tf.keras.activations.relu(x)
            elif self.activation == 'tanh':
                x = tf.keras.activations.tanh(x)
            else:
                raise ValueError("Activación no soportada")
        x = self.classifier(x)
        return tf.keras.activations.softmax(x)

Funciones para entrenar, evaluar y realizar los experimentos

In [16]:
def train_model(model, X_train, y_train, X_val, y_val, loss_fn, epochs=10, batch_size=32):

    # Entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss=loss_fn,
        metrics=['accuracy']
    )

    # Detiene el proceso tras 5 epocas sin encontrar una mejora. Evalúa la pérdida en el
    # conjunto de validación.
    early_stopping = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True,
      verbose=1
    )

    final_model = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        # verbose=0
    )
    return final_model

def evaluate_model(model, X_test, y_test, class_names):
    y_pred_probs = model(X_test, training=False).numpy()
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Si y_test es one-hot, convierte a etiquetas enteras
    if len(y_test.shape) > 1 and y_test.shape[1] > 1:
        y_test_int = np.argmax(y_test, axis=1)
    else:
        y_test_int = y_test

    acc_total = accuracy_score(y_test_int, y_pred)

    acc_per_class = {}
    for cls in np.unique(y_test_int):
        idx = y_test_int == cls
        acc = accuracy_score(y_test_int[idx], y_pred[idx])
        acc_per_class[class_names[cls]] = acc

    cm = confusion_matrix(y_test_int, y_pred)

    return acc_total, acc_per_class, cm


def experiment(X_train, y_train, X_val, y_val, X_test, y_test, layers_size, n_classes, activation, loss_fn, class_names, epochs=10, batch_size=None, n_experiments=5):
    acc_totals = []
    acc_classes_list = []
    cm_list = []

    for i in range(n_experiments):
        print(f"\nEntrenamiento número {i+1}")
        # Crear modelo nuevo para reinicializar pesos
        model = MLP(layers_size, n_classes, activation)
        model.build(input_shape=(None, X_train.shape[1]))

        # Entrenar
        train_model(model, X_train, y_train, X_val, y_val, loss_fn, epochs, batch_size)

        # Evaluar
        acc_total, acc_per_class, cm = evaluate_model(model, X_test, y_test, class_names)
        acc_totals.append(acc_total)
        acc_classes_list.append(acc_per_class)
        cm_list.append(cm)

    return acc_totals, acc_classes_list, cm_list

Experimentos para QuickDraw-10

In [17]:
clases = ['bandage', 'blackberry', 'castle', 'flashlight', 'lion', 'remote-control', 'sink', 'spreadsheet', 'teapot', 'trombone']

In [18]:
# Modelo 1 - QuickDraw-10
# Detalles:
# Activacion: sigmoid
# Perdida: CrossEntropy
# Capas: 3
# Epocas: 100
# Batch size: 100

acc_total_1, acc_clase_1, cm_1 = experiment(
        train_images_10,
        train_labels_10,
        val_image_10,
        val_labels_10,
        test_images_10,
        test_labels_10,
        layers_size=[512, 256,128],
        n_classes=10,
        activation='sigmoid',
        loss_fn='categorical_crossentropy',
        class_names=clases,
        epochs=100,
        batch_size=100,
        n_experiments=5
        )


Entrenamiento número 1


c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 18s 196ms/step - accuracy: 0.0941 - loss: 2.4282 - val_accuracy: 0.1172 - val_loss: 2.3005
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.1131 - loss: 2.3006 - val_accuracy: 0.1091 - val_loss: 2.2977
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 163ms/step - accuracy: 0.1223 - loss: 2.2976 - val_accuracy: 0.1551 - val_loss: 2.2945
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.1426 - loss: 2.2940 - val_accuracy: 0.1369 - val_loss: 2.2912
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 162ms/step - accuracy: 0.1563 - loss: 2.2898 - val_accuracy: 0.1782 - val_loss: 2.2876
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 162ms/step - accuracy: 0.1643 - loss: 2.2857 - val_accuracy: 0.2751 - val_loss: 2.2819
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.2041 - loss: 2.2806 - val_accuracy: 0.2270 - val_loss: 2.2778
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 165ms/step - accuracy: 0.2076 - loss: 2.2747 - 

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 11s 115ms/step - accuracy: 0.1037 - loss: 2.3359 - val_accuracy: 0.1023 - val_loss: 2.2999
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.1129 - loss: 2.2998 - val_accuracy: 0.1111 - val_loss: 2.2968
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 9s 112ms/step - accuracy: 0.1434 - loss: 2.2963 - val_accuracy: 0.1084 - val_loss: 2.2924
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.1297 - loss: 2.2910 - val_accuracy: 0.1897 - val_loss: 2.2873
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.1612 - loss: 2.2861 - val_accuracy: 0.2297 - val_loss: 2.2834
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - accuracy: 0.2093 - loss: 2.2821 - val_accuracy: 0.1714 - val_loss: 2.2793
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 9s 112ms/step - accuracy: 0.2272 - loss: 2.2778 - val_accuracy: 0.1863 - val_loss: 2.2734
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 9s 111ms/step - accuracy: 0.2469 - loss: 2.2710 - val

...

Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 146ms/step - accuracy: 0.0921 - loss: 2.3545 - val_accuracy: 0.1003 - val_loss: 2.2986
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 12s 145ms/step - accuracy: 0.1177 - loss: 2.2979 - val_accuracy: 0.1077 - val_loss: 2.2952
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 12s 143ms/step - accuracy: 0.1369 - loss: 2.2943 - val_accuracy: 0.1016 - val_loss: 2.2908
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 12s 145ms/step - accuracy: 0.1368 - loss: 2.2892 - val_accuracy: 0.2127 - val_loss: 2.2861
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 12s 146ms/step - accuracy: 0.1573 - loss: 2.2847 - val_accuracy: 0.1064 - val_loss: 2.2822
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 148ms/step - accuracy: 0.1864 - loss: 2.2797 - val_accuracy: 0.1579 - val_loss: 2.2745
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 148ms/step - accuracy: 0.1991 - loss: 2.2734 - val_accuracy: 0.3103 - val_loss: 2.2683
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 12s 145ms/step - accuracy: 0.2592 - loss: 2.2663 - 

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 15s 164ms/step - accuracy: 0.1077 - loss: 2.3737 - val_accuracy: 0.1375 - val_loss: 2.2996
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.1346 - loss: 2.2987 - val_accuracy: 0.1287 - val_loss: 2.2954
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.1440 - loss: 2.2950 - val_accuracy: 0.1138 - val_loss: 2.2921
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 159ms/step - accuracy: 0.1496 - loss: 2.2915 - val_accuracy: 0.2690 - val_loss: 2.2873
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 159ms/step - accuracy: 0.1831 - loss: 2.2863 - val_accuracy: 0.1416 - val_loss: 2.2831
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 159ms/step - accuracy: 0.1869 - loss: 2.2815 - val_accuracy: 0.1999 - val_loss: 2.2786
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 160ms/step - accuracy: 0.1928 - loss: 2.2766 - val_accuracy: 0.2073 - val_loss: 2.2722
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 160ms/step - accuracy: 0.2294 - loss: 2.2703 - 

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 15s 165ms/step - accuracy: 0.1026 - loss: 2.3571 - val_accuracy: 0.1009 - val_loss: 2.2985
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.1239 - loss: 2.2979 - val_accuracy: 0.1294 - val_loss: 2.2953
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - accuracy: 0.1175 - loss: 2.2951 - val_accuracy: 0.1341 - val_loss: 2.2916
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 163ms/step - accuracy: 0.1615 - loss: 2.2909 - val_accuracy: 0.2114 - val_loss: 2.2871
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 163ms/step - accuracy: 0.1777 - loss: 2.2869 - val_accuracy: 0.1511 - val_loss: 2.2818
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 162ms/step - accuracy: 0.1856 - loss: 2.2810 - val_accuracy: 0.2337 - val_loss: 2.2777
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 162ms/step - accuracy: 0.2141 - loss: 2.2762 - val_accuracy: 0.2873 - val_loss: 2.2728
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.2450 - loss: 2.2709 - 

In [27]:
acc_clase_1

[{'bandage': 0.8,
  'blackberry': 0.4918032786885246,
  'castle': 0.31297709923664124,
  'flashlight': 0.5086206896551724,
  'lion': 0.5904761904761905,
  'remote-control': 0.7281553398058253,
  'sink': 0.4827586206896552,
  'spreadsheet': 0.6198347107438017,
  'teapot': 0.823076923076923,
  'trombone': 0.2647058823529412},
 {'bandage': 0.5416666666666666,
  'blackberry': 0.8114754098360656,
  'castle': 0.366412213740458,
  'flashlight': 0.8189655172413793,
  'lion': 0.5904761904761905,
  'remote-control': 0.5242718446601942,
  'sink': 0.5344827586206896,
  'spreadsheet': 0.512396694214876,
  'teapot': 0.7307692307692307,
  'trombone': 0.6078431372549019},
 {'bandage': 0.5666666666666667,
  'blackberry': 0.6311475409836066,
  'castle': 0.48854961832061067,
  'flashlight': 0.75,
  'lion': 0.44761904761904764,
  'remote-control': 0.6310679611650486,
  'sink': 0.4051724137931034,
  'spreadsheet': 0.7933884297520661,
  'teapot': 0.7307692307692307,
  'trombone': 0.4215686274509804},
 {'ban

In [28]:
acc_total_1

[0.5643224699828473,
 0.6037735849056604,
 0.5909090909090909,
 0.6423670668953688,
 0.6337907375643225]

In [29]:
cm_1

[array([[ 96,   1,   2,   1,   7,   6,   0,   2,   5,   0],
        [ 31,  60,   5,   5,   7,   1,   0,   3,   8,   2],
        [ 43,   0,  41,   2,   7,  21,   6,   3,   3,   5],
        [ 17,   4,   2,  59,  17,   0,   1,  10,   6,   0],
        [  2,   0,   0,   0,  62,   0,   0,  20,  20,   1],
        [  9,   0,   1,   0,   5,  75,   8,   1,   3,   1],
        [ 12,   6,  12,   1,   1,  22,  56,   3,   1,   2],
        [  1,   1,   0,   1,  28,   6,   2,  75,   2,   5],
        [  6,   3,   0,   0,   9,   4,   0,   1, 107,   0],
        [  3,   3,   2,   2,  30,   3,   3,  26,   3,  27]]),
 array([[65, 18,  1, 20,  2,  3,  3,  1,  1,  6],
        [ 4, 99,  0,  9,  4,  0,  1,  1,  2,  2],
        [ 3, 21, 48, 16,  3,  3, 13,  0,  4, 20],
        [ 1, 12,  2, 95,  4,  0,  0,  1,  1,  0],
        [ 1,  5,  0, 10, 62,  0,  0,  6,  9, 12],
        [ 3,  4,  4,  1,  5, 54, 23,  0,  4,  5],
        [ 1, 24, 11,  2,  2,  7, 62,  0,  0,  7],
        [ 0,  3,  1, 13, 21,  3,  3, 62,  2, 13]

In [21]:
# Modelo 2 - QuickDraw-10
# Detalles:
# Activacion: sigmoid
# Perdida: CrossEntropy
# Capas: 3
# Epocas: 20
# Batch size: 1000

acc_total_2, acc_clase_2, cm_2 = experiment(
        train_images_10,
        train_labels_10,
        val_image_10,
        val_labels_10,
        test_images_10,
        test_labels_10,
        layers_size=[512, 256, 128],
        n_classes=10,
        activation='sigmoid',
        loss_fn='categorical_crossentropy',
        class_names=clases,
        epochs=20,
        batch_size=1000,
        n_experiments=5
        )


Entrenamiento número 1


c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_5', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 585ms/step - accuracy: 0.1014 - loss: 2.4620 - val_accuracy: 0.1009 - val_loss: 2.3749
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 439ms/step - accuracy: 0.1037 - loss: 2.3591 - val_accuracy: 0.1009 - val_loss: 2.3331
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 426ms/step - accuracy: 0.1027 - loss: 2.3284 - val_accuracy: 0.1009 - val_loss: 2.3161
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 426ms/step - accuracy: 0.1030 - loss: 2.3145 - val_accuracy: 0.1043 - val_loss: 2.3079
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 429ms/step - accuracy: 0.1068 - loss: 2.3071 - val_accuracy: 0.1009 - val_loss: 2.3042
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 428ms/step - accuracy: 0.1056 - loss: 2.3033 - val_accuracy: 0.1098 - val_loss: 2.3020
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 428ms/step - accuracy: 0.1135 - loss: 2.3021 - val_accuracy: 0.1064 - val_loss: 2.3008
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 452ms/step - accuracy: 0.1009 - loss: 2.3003 - val_accuracy: 0.1220 - val_loss:

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_6', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 453ms/step - accuracy: 0.0994 - loss: 2.4716 - val_accuracy: 0.0982 - val_loss: 2.3816
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 425ms/step - accuracy: 0.0962 - loss: 2.3684 - val_accuracy: 0.0976 - val_loss: 2.3359
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 425ms/step - accuracy: 0.1009 - loss: 2.3313 - val_accuracy: 0.1009 - val_loss: 2.3165
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 431ms/step - accuracy: 0.1085 - loss: 2.3150 - val_accuracy: 0.1118 - val_loss: 2.3077
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 429ms/step - accuracy: 0.1203 - loss: 2.3068 - val_accuracy: 0.1538 - val_loss: 2.3034
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 429ms/step - accuracy: 0.1341 - loss: 2.3035 - val_accuracy: 0.1341 - val_loss: 2.3013
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 436ms/step - accuracy: 0.1358 - loss: 2.3016 - val_accuracy: 0.1463 - val_loss: 2.3001
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 428ms/step - accuracy: 0.1167 - loss: 2.3004 - val_accuracy: 0.1355 - val_loss:

...

Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 484ms/step - accuracy: 0.1012 - loss: 2.4345 - val_accuracy: 0.0996 - val_loss: 2.3843
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 441ms/step - accuracy: 0.0986 - loss: 2.3716 - val_accuracy: 0.0996 - val_loss: 2.3466
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 436ms/step - accuracy: 0.0977 - loss: 2.3443 - val_accuracy: 0.0989 - val_loss: 2.3260
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 433ms/step - accuracy: 0.1043 - loss: 2.3239 - val_accuracy: 0.1064 - val_loss: 2.3140
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 468ms/step - accuracy: 0.1050 - loss: 2.3123 - val_accuracy: 0.1077 - val_loss: 2.3078
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 439ms/step - accuracy: 0.1165 - loss: 2.3061 - val_accuracy: 0.1220 - val_loss: 2.3041
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 441ms/step - accuracy: 0.1356 - loss: 2.3037 - val_accuracy: 0.1165 - val_loss: 2.3023
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 446ms/step - accuracy: 0.1241 - loss: 2.3015 - val_accuracy: 0.1152 - val_loss:

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_8', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 482ms/step - accuracy: 0.1026 - loss: 2.5686 - val_accuracy: 0.1016 - val_loss: 2.3756
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 464ms/step - accuracy: 0.1016 - loss: 2.3613 - val_accuracy: 0.1016 - val_loss: 2.3267
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 434ms/step - accuracy: 0.1008 - loss: 2.3221 - val_accuracy: 0.1016 - val_loss: 2.3115
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 442ms/step - accuracy: 0.1018 - loss: 2.3087 - val_accuracy: 0.1016 - val_loss: 2.3057
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 437ms/step - accuracy: 0.1021 - loss: 2.3053 - val_accuracy: 0.1016 - val_loss: 2.3030
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 442ms/step - accuracy: 0.1063 - loss: 2.3021 - val_accuracy: 0.1016 - val_loss: 2.3015
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 438ms/step - accuracy: 0.1005 - loss: 2.3013 - val_accuracy: 0.0996 - val_loss: 2.3008
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 437ms/step - accuracy: 0.1071 - loss: 2.3005 - val_accuracy: 0.1084 - val_loss:

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_9', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 501ms/step - accuracy: 0.1004 - loss: 2.4242 - val_accuracy: 0.1016 - val_loss: 2.3608
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 452ms/step - accuracy: 0.1029 - loss: 2.3526 - val_accuracy: 0.1016 - val_loss: 2.3294
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 444ms/step - accuracy: 0.1003 - loss: 2.3274 - val_accuracy: 0.1016 - val_loss: 2.3142
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 441ms/step - accuracy: 0.1016 - loss: 2.3138 - val_accuracy: 0.1023 - val_loss: 2.3062
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 446ms/step - accuracy: 0.1069 - loss: 2.3075 - val_accuracy: 0.1938 - val_loss: 2.3023
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 437ms/step - accuracy: 0.1791 - loss: 2.3013 - val_accuracy: 0.1579 - val_loss: 2.3003
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 442ms/step - accuracy: 0.1643 - loss: 2.2998 - val_accuracy: 0.1477 - val_loss: 2.2990
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 440ms/step - accuracy: 0.1709 - loss: 2.2987 - val_accuracy: 0.2209 - val_loss:

In [30]:
acc_clase_2

[{'bandage': 0.16666666666666666,
  'blackberry': 0.0,
  'castle': 0.0,
  'flashlight': 0.0,
  'lion': 1.0,
  'remote-control': 0.11650485436893204,
  'sink': 0.0,
  'spreadsheet': 0.0,
  'teapot': 0.0,
  'trombone': 0.0},
 {'bandage': 0.0,
  'blackberry': 0.00819672131147541,
  'castle': 0.0,
  'flashlight': 0.0,
  'lion': 0.0,
  'remote-control': 0.04854368932038835,
  'sink': 0.04310344827586207,
  'spreadsheet': 0.1322314049586777,
  'teapot': 0.06153846153846154,
  'trombone': 0.9607843137254902},
 {'bandage': 0.008333333333333333,
  'blackberry': 0.04918032786885246,
  'castle': 0.0,
  'flashlight': 0.0,
  'lion': 0.5904761904761905,
  'remote-control': 0.6893203883495146,
  'sink': 0.07758620689655173,
  'spreadsheet': 0.32231404958677684,
  'teapot': 0.18461538461538463,
  'trombone': 0.09803921568627451},
 {'bandage': 0.075,
  'blackberry': 0.0,
  'castle': 0.0,
  'flashlight': 0.07758620689655173,
  'lion': 0.24761904761904763,
  'remote-control': 0.14563106796116504,
  'sink

In [31]:
acc_total_2

[0.11749571183533447,
 0.11406518010291596,
 0.19039451114922812,
 0.12864493996569468,
 0.15437392795883362]

In [32]:
cm_2

[array([[ 20,   0,   0,   0,  99,   1,   0,   0,   0,   0],
        [ 12,   0,   0,   1, 108,   0,   0,   0,   0,   1],
        [  8,   0,   0,   0, 121,   2,   0,   0,   0,   0],
        [  9,   0,   0,   0, 106,   0,   0,   0,   0,   1],
        [  0,   0,   0,   0, 105,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,  91,  12,   0,   0,   0,   0],
        [  6,   0,   0,   0, 106,   4,   0,   0,   0,   0],
        [  0,   0,   0,   0, 120,   1,   0,   0,   0,   0],
        [  3,   0,   0,   0, 127,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0, 102,   0,   0,   0,   0,   0]]),
 array([[  0,   0,   0,   0,   0,   0,   1,   6,   0, 113],
        [  0,   1,   0,   0,   0,   0,   0,   5,   1, 115],
        [  2,   0,   0,   0,   0,   0,   3,   2,   0, 124],
        [  0,   0,   0,   0,   0,   0,   0,   1,   0, 115],
        [  0,   0,   0,   0,   0,   0,   0,   9,   0,  96],
        [  2,   0,   0,   0,   0,   5,   8,   0,   0,  88],
        [  0,   0,   0,   0,   0,   1,

In [22]:
# Modelo 3 - QuickDraw-10
# Detalles:
# Activacion: relu
# Perdida: Categorical Hinge
# Capas: 4
# Epocas: 60
# Batch size: 550

acc_total_3, acc_clase_3, cm_3 = experiment(
        train_images_10,
        train_labels_10,
        val_image_10,
        val_labels_10,
        test_images_10,
        test_labels_10,
        layers_size=[512, 256, 128, 64],
        n_classes=10,
        activation='relu',
        loss_fn='categorical_hinge',
        class_names=clases,
        epochs=60,
        batch_size=550,
        n_experiments=5
        )


Entrenamiento número 1


c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_10', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 318ms/step - accuracy: 0.0924 - loss: 1.4827 - val_accuracy: 0.0962 - val_loss: 1.1270
Epoch 2/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 298ms/step - accuracy: 0.1015 - loss: 1.1392 - val_accuracy: 0.1091 - val_loss: 1.0366
Epoch 3/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 292ms/step - accuracy: 0.1066 - loss: 1.0329 - val_accuracy: 0.1253 - val_loss: 1.0164
Epoch 4/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 287ms/step - accuracy: 0.1006 - loss: 1.0207 - val_accuracy: 0.1030 - val_loss: 1.0156
Epoch 5/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 292ms/step - accuracy: 0.1150 - loss: 1.0117 - val_accuracy: 0.1321 - val_loss: 1.0050
Epoch 6/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 292ms/step - accuracy: 0.1291 - loss: 1.0058 - val_accuracy: 0.1240 - val_loss: 1.0068
Epoch 7/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 292ms/step - accuracy: 0.1480 - loss: 1.0049 - val_accuracy: 0.1558 - val_loss: 1.0034
Epoch 8/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 291ms/step - accuracy: 0.1593 - loss: 1.0036 - val_accuracy: 0.

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_11', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 338ms/step - accuracy: 0.1031 - loss: 1.3406 - val_accuracy: 0.0982 - val_loss: 1.2591
Epoch 2/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 299ms/step - accuracy: 0.0936 - loss: 1.1180 - val_accuracy: 0.1037 - val_loss: 1.0363
Epoch 3/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 298ms/step - accuracy: 0.0987 - loss: 1.0278 - val_accuracy: 0.1240 - val_loss: 1.0128
Epoch 4/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 302ms/step - accuracy: 0.1198 - loss: 1.0154 - val_accuracy: 0.1131 - val_loss: 1.0177
Epoch 5/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 298ms/step - accuracy: 0.1219 - loss: 1.0132 - val_accuracy: 0.1253 - val_loss: 1.0120
Epoch 6/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 309ms/step - accuracy: 0.1233 - loss: 1.0099 - val_accuracy: 0.1240 - val_loss: 1.0064
Epoch 7/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 309ms/step - accuracy: 0.1278 - loss: 1.0081 - val_accuracy: 0.1389 - val_loss: 1.0070
Epoch 8/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 300ms/step - accuracy: 0.1485 - loss: 1.0057 - val_accuracy: 0.

...

Epoch 1/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 329ms/step - accuracy: 0.0987 - loss: 1.2833 - val_accuracy: 0.0982 - val_loss: 1.1913
Epoch 2/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 306ms/step - accuracy: 0.0986 - loss: 1.0772 - val_accuracy: 0.1037 - val_loss: 1.0343
Epoch 3/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 297ms/step - accuracy: 0.1085 - loss: 1.0214 - val_accuracy: 0.1070 - val_loss: 1.0212
Epoch 4/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 302ms/step - accuracy: 0.0970 - loss: 1.0200 - val_accuracy: 0.1064 - val_loss: 1.0091
Epoch 5/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 299ms/step - accuracy: 0.1049 - loss: 1.0099 - val_accuracy: 0.1206 - val_loss: 1.0088
Epoch 6/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 291ms/step - accuracy: 0.1343 - loss: 1.0073 - val_accuracy: 0.1538 - val_loss: 1.0064
Epoch 7/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 298ms/step - accuracy: 0.1374 - loss: 1.0063 - val_accuracy: 0.1511 - val_loss: 1.0058
Epoch 8/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 295ms/step - accuracy: 0.1440 - loss: 1.0060 - val_accuracy: 0.

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_13', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 318ms/step - accuracy: 0.0974 - loss: 1.1894 - val_accuracy: 0.0928 - val_loss: 1.1350
Epoch 2/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 297ms/step - accuracy: 0.0929 - loss: 1.1191 - val_accuracy: 0.0989 - val_loss: 1.0290
Epoch 3/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 296ms/step - accuracy: 0.1024 - loss: 1.0262 - val_accuracy: 0.1043 - val_loss: 1.0142
Epoch 4/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 296ms/step - accuracy: 0.0989 - loss: 1.0162 - val_accuracy: 0.1043 - val_loss: 1.0088
Epoch 5/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 304ms/step - accuracy: 0.1043 - loss: 1.0110 - val_accuracy: 0.0996 - val_loss: 1.0086
Epoch 6/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 302ms/step - accuracy: 0.1127 - loss: 1.0106 - val_accuracy: 0.1084 - val_loss: 1.0072
Epoch 7/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 299ms/step - accuracy: 0.1252 - loss: 1.0059 - val_accuracy: 0.1009 - val_loss: 1.0124
Epoch 8/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 295ms/step - accuracy: 0.1235 - loss: 1.0071 - val_accuracy: 0.

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_14', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 310ms/step - accuracy: 0.1007 - loss: 1.2592 - val_accuracy: 0.0982 - val_loss: 1.1239
Epoch 2/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 310ms/step - accuracy: 0.0983 - loss: 1.0894 - val_accuracy: 0.0942 - val_loss: 1.0293
Epoch 3/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 295ms/step - accuracy: 0.1023 - loss: 1.0317 - val_accuracy: 0.0996 - val_loss: 1.0178
Epoch 4/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 305ms/step - accuracy: 0.1016 - loss: 1.0156 - val_accuracy: 0.1314 - val_loss: 1.0115
Epoch 5/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 297ms/step - accuracy: 0.1285 - loss: 1.0085 - val_accuracy: 0.0982 - val_loss: 1.0064
Epoch 6/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 296ms/step - accuracy: 0.1196 - loss: 1.0071 - val_accuracy: 0.1213 - val_loss: 1.0071
Epoch 7/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 296ms/step - accuracy: 0.1268 - loss: 1.0080 - val_accuracy: 0.1565 - val_loss: 1.0068
Epoch 8/60
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 294ms/step - accuracy: 0.1323 - loss: 1.0064 - val_accuracy: 0.

In [33]:
acc_clase_3

[{'bandage': 0.5,
  'blackberry': 0.09836065573770492,
  'castle': 0.022900763358778626,
  'flashlight': 0.034482758620689655,
  'lion': 0.0,
  'remote-control': 0.8446601941747572,
  'sink': 0.3793103448275862,
  'spreadsheet': 0.2809917355371901,
  'teapot': 0.46153846153846156,
  'trombone': 0.2549019607843137},
 {'bandage': 0.2,
  'blackberry': 0.3442622950819672,
  'castle': 0.04580152671755725,
  'flashlight': 0.16379310344827586,
  'lion': 0.3523809523809524,
  'remote-control': 0.8155339805825242,
  'sink': 0.08620689655172414,
  'spreadsheet': 0.4132231404958678,
  'teapot': 0.5307692307692308,
  'trombone': 0.12745098039215685},
 {'bandage': 0.16666666666666666,
  'blackberry': 0.5983606557377049,
  'castle': 0.09923664122137404,
  'flashlight': 0.25,
  'lion': 0.0,
  'remote-control': 0.8252427184466019,
  'sink': 0.3448275862068966,
  'spreadsheet': 0.0,
  'teapot': 0.23846153846153847,
  'trombone': 0.19607843137254902},
 {'bandage': 0.03333333333333333,
  'blackberry': 0.

In [34]:
acc_total_3

[0.2830188679245283,
 0.30360205831903947,
 0.2667238421955403,
 0.3241852487135506,
 0.3259005145797599]

In [35]:
cm_3

[array([[60,  1,  1,  0,  1, 28, 20,  1,  4,  4],
        [57, 12,  1,  1,  1,  4, 21,  9,  5, 11],
        [27,  0,  3,  1,  0, 48, 46,  1,  1,  4],
        [61,  1,  1,  4,  0,  2, 23,  5,  0, 19],
        [35,  0,  0,  1,  0, 16, 13, 11,  7, 22],
        [ 4,  1,  0,  0,  0, 87,  7,  0,  2,  2],
        [15,  2,  2,  0,  0, 49, 44,  2,  1,  1],
        [30,  0,  0,  1,  1, 14, 10, 34,  4, 27],
        [24,  0,  0,  1,  0, 35,  6,  4, 60,  0],
        [18,  0,  1,  0,  0, 11, 30, 14,  2, 26]]),
 array([[24,  4,  0, 11, 20, 25,  0, 13, 21,  2],
        [ 6, 42,  3, 16, 16,  5,  0, 16,  9,  9],
        [21,  4,  6, 11,  9, 45,  5,  7, 12, 11],
        [ 3, 18,  0, 19, 27,  3,  0, 25, 17,  4],
        [ 0,  5,  1,  2, 37, 12,  0, 23, 20,  5],
        [ 4,  0,  0,  1,  5, 84,  1,  1,  4,  3],
        [ 9,  9,  5, 11,  4, 53, 10,  4,  4,  7],
        [ 1,  5,  1,  2, 35, 11,  0, 50, 13,  3],
        [ 1,  3,  1,  1, 13, 32,  0,  8, 69,  2],
        [ 3,  4,  1,  5, 26, 12,  0, 31,  7, 13]

Experimentos para QuickDraw-Animals

In [23]:
clases = ['sheep', 'bear', 'bee', 'cat', 'camel', 'cow', 'crab', 'crocodile', 'duck', 'elephant', 'dog', 'giraffe']

In [24]:
# Modelo 1 - QuickDraw-Animals
# Detalles:
# Activacion: tanh
# Perdida: categorical_hinge
# Capas: 2
# Epocas: 60
# Batch size: 550

acc_total_4, acc_clase_4, cm_4 = experiment(
        train_images_animals,
        train_labels_animals,
        val_images_animals,
        val_labels_animals,
        test_images_animals,
        test_labels_animals,
        layers_size=[512, 256],
        n_classes=12,
        activation='tanh',
        loss_fn='categorical_hinge',
        class_names=clases,
        epochs=10,
        batch_size=1000,
        n_experiments=5
        )


Entrenamiento número 1


c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_15', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 852ms/step - accuracy: 0.0836 - loss: 1.5067 - val_accuracy: 0.0833 - val_loss: 1.1727
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 469ms/step - accuracy: 0.0819 - loss: 1.1416 - val_accuracy: 0.0861 - val_loss: 1.0924
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 460ms/step - accuracy: 0.0778 - loss: 1.1055 - val_accuracy: 0.0833 - val_loss: 1.0925
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 461ms/step - accuracy: 0.0833 - loss: 1.0915 - val_accuracy: 0.0822 - val_loss: 1.0794
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 468ms/step - accuracy: 0.0838 - loss: 1.0781 - val_accuracy: 0.0833 - val_loss: 1.0823
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 461ms/step - accuracy: 0.0861 - loss: 1.0710 - val_accuracy: 0.0833 - val_loss: 1.0564
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 468ms/step - accuracy: 0.0849 - loss: 1.0538 - val_accuracy: 0.0833 - val_loss: 1.0456
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 462ms/step - accuracy: 0.0785 - loss: 1.0496 - val_accuracy: 0

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_16', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 482ms/step - accuracy: 0.0858 - loss: 1.1617 - val_accuracy: 0.0828 - val_loss: 1.0452
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 443ms/step - accuracy: 0.0904 - loss: 1.0422 - val_accuracy: 0.0867 - val_loss: 1.0255
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 438ms/step - accuracy: 0.0888 - loss: 1.0302 - val_accuracy: 0.0817 - val_loss: 1.0301
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 441ms/step - accuracy: 0.0829 - loss: 1.0342 - val_accuracy: 0.0833 - val_loss: 1.0304
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 441ms/step - accuracy: 0.0802 - loss: 1.0322 - val_accuracy: 0.0828 - val_loss: 1.0371
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 443ms/step - accuracy: 0.0811 - loss: 1.0346 - val_accuracy: 0.0833 - val_loss: 1.0337
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 440ms/step - accuracy: 0.0837 - loss: 1.0333 - val_accuracy: 0.0833 - val_loss: 1.0365
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 444ms/step - accuracy: 0.0858 - loss: 1.0352 - val_accuracy: 0.

...

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 496ms/step - accuracy: 0.0906 - loss: 1.2832 - val_accuracy: 0.0833 - val_loss: 1.1112
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 480ms/step - accuracy: 0.0828 - loss: 1.0777 - val_accuracy: 0.0833 - val_loss: 1.0598
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 468ms/step - accuracy: 0.0823 - loss: 1.0603 - val_accuracy: 0.0833 - val_loss: 1.0605
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 462ms/step - accuracy: 0.0895 - loss: 1.0621 - val_accuracy: 0.0828 - val_loss: 1.0595
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 445ms/step - accuracy: 0.0836 - loss: 1.0576 - val_accuracy: 0.0833 - val_loss: 1.0514
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 460ms/step - accuracy: 0.0852 - loss: 1.0493 - val_accuracy: 0.0739 - val_loss: 1.0444
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 463ms/step - accuracy: 0.0747 - loss: 1.0463 - val_accuracy: 0.0833 - val_loss: 1.0356
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 463ms/step - accuracy: 0.0823 - loss: 1.0388 - val_accuracy: 0.

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_18', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 493ms/step - accuracy: 0.0839 - loss: 1.2707 - val_accuracy: 0.0833 - val_loss: 1.1414
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 462ms/step - accuracy: 0.0814 - loss: 1.1241 - val_accuracy: 0.0833 - val_loss: 1.0663
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 461ms/step - accuracy: 0.0848 - loss: 1.0576 - val_accuracy: 0.0844 - val_loss: 1.0381
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 462ms/step - accuracy: 0.0933 - loss: 1.0352 - val_accuracy: 0.0833 - val_loss: 1.0348
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 465ms/step - accuracy: 0.0837 - loss: 1.0369 - val_accuracy: 0.0839 - val_loss: 1.0367
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 465ms/step - accuracy: 0.0797 - loss: 1.0370 - val_accuracy: 0.0833 - val_loss: 1.0334
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 458ms/step - accuracy: 0.0844 - loss: 1.0324 - val_accuracy: 0.0833 - val_loss: 1.0343
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 462ms/step - accuracy: 0.0838 - loss: 1.0331 - val_accuracy: 0.

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_19', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 530ms/step - accuracy: 0.0827 - loss: 1.2412 - val_accuracy: 0.0833 - val_loss: 1.1277
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 494ms/step - accuracy: 0.0843 - loss: 1.0952 - val_accuracy: 0.0950 - val_loss: 1.0371
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 476ms/step - accuracy: 0.0882 - loss: 1.0399 - val_accuracy: 0.0833 - val_loss: 1.0447
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 476ms/step - accuracy: 0.0909 - loss: 1.0402 - val_accuracy: 0.0800 - val_loss: 1.0318
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 476ms/step - accuracy: 0.0867 - loss: 1.0370 - val_accuracy: 0.0833 - val_loss: 1.0382
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 483ms/step - accuracy: 0.0860 - loss: 1.0369 - val_accuracy: 0.0833 - val_loss: 1.0339
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 482ms/step - accuracy: 0.0766 - loss: 1.0341 - val_accuracy: 0.0833 - val_loss: 1.0264
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 489ms/step - accuracy: 0.0799 - loss: 1.0275 - val_accuracy: 0.

In [36]:
acc_clase_4

[{'sheep': 0.0,
  'bear': 0.005,
  'bee': 0.465,
  'cat': 0.0,
  'camel': 0.0,
  'cow': 0.54,
  'crab': 0.0,
  'crocodile': 0.0,
  'duck': 0.0,
  'elephant': 0.0,
  'dog': 0.0,
  'giraffe': 0.0},
 {'sheep': 0.0,
  'bear': 0.0,
  'bee': 1.0,
  'cat': 0.0,
  'camel': 0.0,
  'cow': 0.0,
  'crab': 0.0,
  'crocodile': 0.0,
  'duck': 0.0,
  'elephant': 0.0,
  'dog': 0.0,
  'giraffe': 0.0},
 {'sheep': 0.0,
  'bear': 0.0,
  'bee': 0.0,
  'cat': 0.845,
  'camel': 0.0,
  'cow': 0.0,
  'crab': 0.0,
  'crocodile': 0.045,
  'duck': 0.0,
  'elephant': 0.0,
  'dog': 0.0,
  'giraffe': 0.0},
 {'sheep': 0.0,
  'bear': 0.0,
  'bee': 0.0,
  'cat': 0.095,
  'camel': 0.0,
  'cow': 0.0,
  'crab': 0.0,
  'crocodile': 0.0,
  'duck': 0.0,
  'elephant': 0.0,
  'dog': 0.0,
  'giraffe': 0.9949748743718593},
 {'sheep': 0.0,
  'bear': 0.0,
  'bee': 0.0,
  'cat': 0.0,
  'camel': 0.0,
  'cow': 0.0,
  'crab': 0.0,
  'crocodile': 0.0,
  'duck': 1.0,
  'elephant': 0.0,
  'dog': 0.0,
  'giraffe': 0.0}]

In [37]:
acc_total_4

[0.08420175072947061,
 0.08336807002917883,
 0.07419758232596915,
 0.09045435598165902,
 0.08336807002917883]

In [38]:
cm_4

[array([[  0,   0,  84,   0,   0, 116,   0,   0,   0,   0,   0,   0],
        [  0,   1,  78,   0,   0, 121,   0,   0,   0,   0,   0,   0],
        [  0,   3,  93,   0,   0, 104,   0,   0,   0,   0,   0,   0],
        [  0,   1,  83,   0,   0, 116,   0,   0,   0,   0,   0,   0],
        [  0,   1,  63,   0,   0, 136,   0,   0,   0,   0,   0,   0],
        [  0,   1,  91,   0,   0, 108,   0,   0,   0,   0,   0,   0],
        [  0,   0,  80,   0,   0, 120,   0,   0,   0,   0,   0,   0],
        [  0,   1,  91,   0,   0, 108,   0,   0,   0,   0,   0,   0],
        [  0,   0,  72,   0,   0, 128,   0,   0,   0,   0,   0,   0],
        [  0,   1, 101,   0,   0,  98,   0,   0,   0,   0,   0,   0],
        [  0,   0,  75,   0,   0, 125,   0,   0,   0,   0,   0,   0],
        [  0,   2,  44,   0,   0, 153,   0,   0,   0,   0,   0,   0]]),
 array([[  0,   0, 200,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 200,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,

In [25]:
# Modelo 2 - QuickDraw-Animals
# Detalles:
# Activacion: tanh
# Perdida: categorical_crossentropy
# Capas: 2
# Epocas: 10
# Batch size: 1000

acc_total_5, acc_clase_5, cm_5 = experiment(
        train_images_animals,
        train_labels_animals,
        val_images_animals,
        val_labels_animals,
        test_images_animals,
        test_labels_animals,
        layers_size=[512, 256, 128],
        n_classes=12,
        activation='tanh',
        loss_fn='categorical_crossentropy',
        class_names=clases,
        epochs=100,
        batch_size=150,
        n_experiments=5
        )


Entrenamiento número 1


c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_20', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 171ms/step - accuracy: 0.0891 - loss: 2.6282 - val_accuracy: 0.1344 - val_loss: 2.4731
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 162ms/step - accuracy: 0.1130 - loss: 2.4745 - val_accuracy: 0.1667 - val_loss: 2.4235
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 160ms/step - accuracy: 0.1340 - loss: 2.4556 - val_accuracy: 0.1389 - val_loss: 2.4599
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step - accuracy: 0.1348 - loss: 2.4489 - val_accuracy: 0.1894 - val_loss: 2.4429
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 160ms/step - accuracy: 0.1450 - loss: 2.4371 - val_accuracy: 0.0911 - val_loss: 2.4255
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step - accuracy: 0.1410 - loss: 2.4339 - val_accuracy: 0.1456 - val_loss: 2.4135
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 158ms/step - accuracy: 0.1407 - loss: 2.4249 - val_accuracy: 0.2306 - val_loss: 2.3625
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 158ms/step - accuracy: 0.1479 - loss: 2.4266 - 

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_21', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 172ms/step - accuracy: 0.0966 - loss: 2.6077 - val_accuracy: 0.0922 - val_loss: 2.4741
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 165ms/step - accuracy: 0.1091 - loss: 2.4732 - val_accuracy: 0.1139 - val_loss: 2.4521
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 164ms/step - accuracy: 0.1362 - loss: 2.4468 - val_accuracy: 0.0883 - val_loss: 2.4409
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 166ms/step - accuracy: 0.1252 - loss: 2.4442 - val_accuracy: 0.1044 - val_loss: 2.4333
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 165ms/step - accuracy: 0.1425 - loss: 2.4257 - val_accuracy: 0.1739 - val_loss: 2.4259
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 165ms/step - accuracy: 0.1477 - loss: 2.4225 - val_accuracy: 0.2261 - val_loss: 2.3861
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 167ms/step - accuracy: 0.1630 - loss: 2.4234 - val_accuracy: 0.1194 - val_loss: 2.4351
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 11s 167ms/step - accuracy: 0.1398 - loss: 2.4208 - 

...

Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 14s 187ms/step - accuracy: 0.0839 - loss: 2.6437 - val_accuracy: 0.0911 - val_loss: 2.4777
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 181ms/step - accuracy: 0.0993 - loss: 2.4800 - val_accuracy: 0.0900 - val_loss: 2.4722
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 181ms/step - accuracy: 0.1107 - loss: 2.4587 - val_accuracy: 0.1300 - val_loss: 2.4720
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 184ms/step - accuracy: 0.1082 - loss: 2.4615 - val_accuracy: 0.2000 - val_loss: 2.4150
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 181ms/step - accuracy: 0.1334 - loss: 2.4570 - val_accuracy: 0.1811 - val_loss: 2.4342
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 181ms/step - accuracy: 0.1293 - loss: 2.4471 - val_accuracy: 0.0833 - val_loss: 2.4514
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 182ms/step - accuracy: 0.1354 - loss: 2.4352 - val_accuracy: 0.0900 - val_loss: 2.4550
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 181ms/step - accuracy: 0.1428 - loss: 2.4234 - 

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_23', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 14s 188ms/step - accuracy: 0.0929 - loss: 2.6400 - val_accuracy: 0.1089 - val_loss: 2.4901
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 184ms/step - accuracy: 0.0910 - loss: 2.4829 - val_accuracy: 0.0833 - val_loss: 2.4866
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 186ms/step - accuracy: 0.1108 - loss: 2.4675 - val_accuracy: 0.1078 - val_loss: 2.4478
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 193ms/step - accuracy: 0.1095 - loss: 2.4596 - val_accuracy: 0.1189 - val_loss: 2.4406
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 188ms/step - accuracy: 0.1298 - loss: 2.4451 - val_accuracy: 0.0883 - val_loss: 2.4862
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 186ms/step - accuracy: 0.1369 - loss: 2.4435 - val_accuracy: 0.2183 - val_loss: 2.4156
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 185ms/step - accuracy: 0.1406 - loss: 2.4293 - val_accuracy: 0.1028 - val_loss: 2.4692
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 185ms/step - accuracy: 0.1342 - loss: 2.4250 - 

c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_24', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 14s 189ms/step - accuracy: 0.0930 - loss: 2.5813 - val_accuracy: 0.1133 - val_loss: 2.4772
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 183ms/step - accuracy: 0.1113 - loss: 2.4764 - val_accuracy: 0.0894 - val_loss: 2.4823
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 180ms/step - accuracy: 0.1147 - loss: 2.4651 - val_accuracy: 0.1406 - val_loss: 2.4509
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 182ms/step - accuracy: 0.1249 - loss: 2.4518 - val_accuracy: 0.1144 - val_loss: 2.4462
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 182ms/step - accuracy: 0.1414 - loss: 2.4448 - val_accuracy: 0.1333 - val_loss: 2.4446
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 181ms/step - accuracy: 0.1398 - loss: 2.4322 - val_accuracy: 0.0972 - val_loss: 2.4510
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 183ms/step - accuracy: 0.1338 - loss: 2.4362 - val_accuracy: 0.1389 - val_loss: 2.4243
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 181ms/step - accuracy: 0.1485 - loss: 2.4294 - 

In [39]:
acc_clase_5

[{'sheep': 0.71,
  'bear': 0.0,
  'bee': 0.35,
  'cat': 0.0,
  'camel': 0.0,
  'cow': 0.315,
  'crab': 0.05,
  'crocodile': 0.765,
  'duck': 0.0,
  'elephant': 0.04,
  'dog': 0.0,
  'giraffe': 0.0},
 {'sheep': 0.0,
  'bear': 0.005,
  'bee': 0.95,
  'cat': 0.0,
  'camel': 0.0,
  'cow': 0.055,
  'crab': 0.0,
  'crocodile': 0.015,
  'duck': 0.0,
  'elephant': 0.0,
  'dog': 0.0,
  'giraffe': 0.8844221105527639},
 {'sheep': 0.0,
  'bear': 0.0,
  'bee': 0.0,
  'cat': 0.0,
  'camel': 0.005,
  'cow': 0.0,
  'crab': 0.0,
  'crocodile': 0.995,
  'duck': 0.0,
  'elephant': 0.0,
  'dog': 0.0,
  'giraffe': 0.5577889447236181},
 {'sheep': 0.585,
  'bear': 0.235,
  'bee': 0.535,
  'cat': 0.055,
  'camel': 0.0,
  'cow': 0.435,
  'crab': 0.0,
  'crocodile': 0.78,
  'duck': 0.0,
  'elephant': 0.0,
  'dog': 0.0,
  'giraffe': 0.8140703517587939},
 {'sheep': 0.295,
  'bear': 0.025,
  'bee': 0.805,
  'cat': 0.0,
  'camel': 0.02,
  'cow': 0.04,
  'crab': 0.0,
  'crocodile': 0.81,
  'duck': 0.0,
  'elephant':

In [40]:
acc_total_5

[0.18591079616506878,
 0.15881617340558565,
 0.12963734889537307,
 0.2863693205502293,
 0.24510212588578575]

In [41]:
cm_5

[array([[142,   0,   2,   0,   0,  42,   2,   8,   0,   4,   0,   0],
        [108,   0,  61,   0,   0,   9,   0,   8,   0,  14,   0,   0],
        [ 59,   0,  70,   0,   0,  29,   0,  27,   0,  15,   0,   0],
        [121,   0,  26,   0,   0,  17,   3,  20,   0,  13,   0,   0],
        [155,   0,   5,   0,   0,  19,   2,   7,   0,  12,   0,   0],
        [ 90,   0,  10,   0,   0,  63,   6,  27,   0,   4,   0,   0],
        [ 62,   0,  11,   0,   0,  39,  10,  68,   0,  10,   0,   0],
        [ 22,   0,   2,   0,   0,  19,   2, 153,   0,   2,   0,   0],
        [122,   0,  25,   0,   0,  14,   2,   9,   0,  28,   0,   0],
        [109,   0,  20,   0,   0,  37,   2,  24,   0,   8,   0,   0],
        [ 94,   0,  13,   0,   0,  45,   3,  33,   0,  12,   0,   0],
        [ 35,   0, 151,   0,   0,   0,   0,   1,   0,  12,   0,   0]]),
 array([[  0,   0, 138,   0,   0,  24,   0,   0,   0,   6,   0,  32],
        [  0,   1,  93,   0,   0,   5,   0,   0,   0,   1,   0, 100],
        [  0,   0,

In [26]:
# Modelo 3 - QuickDraw-Animals
# Detalles:
# Activacion: sigmoid
# Perdida: kullback_leibler_divergence
# Capas: 2
# Epocas: 10
# Batch size: 1000

acc_total_6, acc_clase_6, cm_6 = experiment(
        train_images_animals,
        train_labels_animals,
        val_images_animals,
        val_labels_animals,
        test_images_animals,
        test_labels_animals,
        layers_size=[512, 256, 128, 64],
        n_classes=12,
        activation='sigmoid',
        loss_fn='kullback_leibler_divergence',
        class_names=clases,
        epochs=10,
        batch_size=20,
        n_experiments=5
        )


Entrenamiento número 1


c:\Users\vicen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'mlp_25', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/10


ValueError: Could not interpret loss identifier: kullback_leibler_divergence